# <a id='toc1_'></a>[II. Data_Unification](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [II. Data_Unification](#toc1_)    
    - [Import Libraries and PyPower.py](#toc1_1_1_)    
    - [Otter DataFrame](#toc1_1_2_)    
      - [Columns marca and local (delete rows)](#toc1_1_2_1_)    
      - [Columns Float Subtotal](#toc1_1_2_2_)    
      - [Rename columns to english names](#toc1_1_2_3_)    
      - [Columns subtotal, vat and discount are object type and they should be float](#toc1_1_2_4_)    
    - [Deliverect DataFrame](#toc1_1_3_)    
        - [For the project, the customer only wants to unify the data from the delivery platforms](#toc1_1_3_1_1_)    
        - [Unify columns names](#toc1_1_3_1_2_)    
    - [DataFrames final unification](#toc1_1_4_)    
      - [Total columns unification](#toc1_1_4_1_)    
      - [Now we order the columns](#toc1_1_4_2_)    
      - [Unification of status column](#toc1_1_4_3_)    
      - [Unification of location](#toc1_1_4_4_)    
    - [Export DataFrame](#toc1_1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_1_'></a>[Import Libraries and PyPower.py](#toc0_)

In [1]:
import pandas as pd
from sys import displayhook
import sys
sys.path.append('../Python')
from  PyPower import *

### <a id='toc1_1_2_'></a>[Otter DataFrame](#toc0_)

In [2]:
otter = pd.read_csv('../../data/1-clean/otter_clean.csv')
otter_original_shape= otter.shape
otter.head()

,fecha,marca,local,canal,id_orden,estado_orden,subtotal,impuesto,descuento
0,2023-11-03 15:55:55,Umbrella SH,EMEA - SP - MAD - Tetuan,justeat,138656463,Nuevo,10,0,0
1,2023-11-03 15:49:40,Umbrella SH,EMEA - SP - MAD - Tetuan,justeat,138656912,Nuevo,"22,4",0,0
2,2023-11-03 15:49:02,Umbrella SH,EMEA - SP - MAD - Tetuan,glovo,MSDGW9UM,Entregado,"10,4",0,0
3,2023-11-03 15:48:41,Umbrella SH,Umbrella SH - Calle Linares,ubereats,8D40B,Nuevo,"36,6",0,"-13,9"
4,2023-11-03 15:44:25,Umbrella SH,Goleta,ubereats,DF131,Entregado,"31,8",0,"-13,9"


In [3]:
otter.describe(include='all')

,fecha,marca,local,canal,id_orden,estado_orden,subtotal,impuesto,descuento
count,534,528,534,534,534,534,534,534,534
unique,530,3,8,4,534,3,253,18,13
top,2023-11-01 20:23:57,Umbrella SH,EMEA - SP - MAD - Tetuan,ubereats,138656463,Entregado,"31,8",0,0
freq,2,443,132,353,1,464,35,517,315


#### <a id='toc1_1_2_1_'></a>[Columns marca and local (delete rows)](#toc0_)
For the scope of the project, we will select only the brands "Tepuy Burger" and "Tepuy Basics", and localization in "Tepuy" only

In [4]:
otter.marca.value_counts()

marca
Umbrella SH     443
Tepuy Burger     60
Tepuy Basics     25
Name: count, dtype: int64

In [5]:
filtered_df = otter[otter['marca'].isin(['Tepuy Burger', 'Tepuy Basics'])]

filtered_df.marca.value_counts()

marca
Tepuy Burger    60
Tepuy Basics    25
Name: count, dtype: int64

In [6]:
otter = filtered_df.copy()

In [7]:
otter.local.value_counts()

local
Tepuy Burger - Rodolfo Salazar          29
Tepuy Burger - Benidorm                 22
Tepuy Burger - Calle Bazan               9
Umbrella SH - San Vicent del Raspeig     9
Goleta                                   8
EMEA - SP - MAD - Salamanca              5
Umbrella SH - Calle Linares              3
Name: count, dtype: int64

In [8]:
filtered_df = otter[otter['local'].isin(['Tepuy Burger - Rodolfo Salazar', 'Tepuy Burger - Benidorm', 'Tepuy Burger - Calle Bazan'])]

filtered_df.local.value_counts()

local
Tepuy Burger - Rodolfo Salazar    29
Tepuy Burger - Benidorm           22
Tepuy Burger - Calle Bazan         9
Name: count, dtype: int64

In [9]:
otter = filtered_df.copy()

#### <a id='toc1_1_2_2_'></a>[Columns Float Subtotal](#toc0_)

In [10]:
#The data is going a SQL database, so we need the numbers with . instead of ,
otter.subtotal = otter.subtotal.apply(replace_to, change_for=',', change_to='.')
otter.impuesto = otter.impuesto.apply(replace_to, change_for=',', change_to='.')
otter.descuento = otter.descuento.apply(replace_to, change_for=',', change_to='.')

In [11]:
otter.describe()

,fecha,marca,local,canal,id_orden,estado_orden,subtotal,impuesto,descuento
count,60,60,60,60,60,60,60,60,60
unique,58,1,3,3,60,3,50,2,3
top,2023-11-01 20:14:55,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,AULRBGNM,Entregado,26.9,0,0
freq,2,60,29,41,1,54,3,59,58


#### <a id='toc1_1_2_3_'></a>[Rename columns to english names](#toc0_)

In [12]:
columns_to_rename={'fecha':'created_datetime',
                   'marca':'brand',
                   'local':'location',
                   'canal':'channel',
                   'id_orden':'order_id_software',
                   'impuesto':'vat',
                   'descuento':'discount',
                   'estado_orden':'status'}

otter = rename_columns(otter, columns_to_rename)


In [13]:
otter.head()

,created_datetime,brand,location,channel,order_id_software,status,subtotal,vat,discount
16,2023-11-03 14:46:46,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,AULRBGNM,Entregado,26.9,0,0
18,2023-11-03 14:40:15,Tepuy Burger,Tepuy Burger - Benidorm,glovo,B1YBHZJW,Entregado,51.5,0,0
22,2023-11-03 14:23:06,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,ALMVLKTB,Entregado,26.9,0,0
50,2023-11-02 22:32:27,Tepuy Burger,Tepuy Burger - Benidorm,glovo,BXVJAQHE,Entregado,40.9,0,0
53,2023-11-02 22:28:20,Tepuy Burger,Tepuy Burger - Calle Bazan,glovo,ABUGGFKZ,Cancelado,20.4,0,0


In [14]:
otter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 16 to 509
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   created_datetime   60 non-null     object
 1   brand              60 non-null     object
 2   location           60 non-null     object
 3   channel            60 non-null     object
 4   order_id_software  60 non-null     object
 5   status             60 non-null     object
 6   subtotal           60 non-null     object
 7   vat                60 non-null     object
 8   discount           60 non-null     object
dtypes: object(9)
memory usage: 4.7+ KB


#### <a id='toc1_1_2_4_'></a>[Columns subtotal, vat and discount are object type and they should be float](#toc0_)

In [15]:
#PyPower method
otter= change_type(otter, 'subtotal', 'object','float')
otter= change_type(otter, 'vat', 'object','float')
otter= change_type(otter, 'discount', 'object','float') 

In [16]:
otter.channel = otter.channel.apply(replace_to, change_for='-', change_to='_')

otter.channel.value_counts()

channel
glovo           41
ubereats        17
gloria_foods     2
Name: count, dtype: int64

In [17]:
#Asign new column to explain from which system is this order comming from

otter['software'] = 'otter'

### <a id='toc1_1_3_'></a>[Deliverect DataFrame](#toc0_)

In [18]:
deliverect = pd.read_csv('../../data/1-clean/deliverect_clean.csv')
deliverect_original_shape= otter.shape
deliverect.head()

,createdtimeutc,location,orderid,channel,status,type,payment,paymentamount,deliverycost,discounttotal,tip,drivertip,subtotal,vat
0,2023-10-31 21:31:15,Alicante - Bazan,138509659,Just Eat,ACCEPTED,DELIVERY,CARD,85.39,1.99,0.0,0.0,0.0,83.4,7.57
1,2023-10-31 21:02:26,Alicante - Periodista Rodolfo Salazar,138502564,Just Eat,DELIVERED,DELIVERY,CARD,41.89,1.99,0.0,0.0,0.0,39.9,3.62
2,2023-10-29 20:20:15,Alicante - Periodista Rodolfo Salazar,138409768,Just Eat,DELIVERED,DELIVERY,CARD,31.09,1.99,0.0,0.0,0.0,29.1,2.65
3,2023-10-29 19:53:10,Alicante - Periodista Rodolfo Salazar,138402233,Just Eat,DELIVERED,DELIVERY,CARD,19.69,1.99,0.0,0.0,0.0,17.7,1.61
4,2023-10-29 19:28:17,Alicante - Periodista Rodolfo Salazar,138390234,Just Eat,DELIVERED,DELIVERY,CARD,15.99,2.99,0.0,0.0,0.0,13.0,1.19


In [19]:
deliverect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   createdtimeutc  62 non-null     object 
 1   location        62 non-null     object 
 2   orderid         62 non-null     int64  
 3   channel         62 non-null     object 
 4   status          62 non-null     object 
 5   type            62 non-null     object 
 6   payment         62 non-null     object 
 7   paymentamount   62 non-null     float64
 8   deliverycost    62 non-null     float64
 9   discounttotal   62 non-null     float64
 10  tip             62 non-null     float64
 11  drivertip       62 non-null     float64
 12  subtotal        62 non-null     float64
 13  vat             62 non-null     float64
dtypes: float64(7), int64(1), object(6)
memory usage: 6.9+ KB


##### <a id='toc1_1_3_1_1_'></a>[For the project, the customer only wants to unify the data from the delivery platforms](#toc0_)
Delete where type is not DELIVERY

In [20]:
filtered_df = deliverect[deliverect['type']=='DELIVERY']

deliverect = filtered_df.copy()

##### <a id='toc1_1_3_1_2_'></a>[Unify columns names](#toc0_)

In [21]:
columns_to_rename={'createdtimeutc':'created_datetime',
                   'payment':'payment_method',
                   'paymentamount':'total',
                   'deliverycost':'delivery_cost',
                   'orderid':'order_id_software',
                   'discounttotal':'discount',
                   'drivertip':'driver_tip'}

deliverect = rename_columns(deliverect, columns_to_rename)


In [22]:
deliverect.head()

,created_datetime,location,order_id_software,channel,status,type,payment_method,total,delivery_cost,discount,tip,driver_tip,subtotal,vat
0,2023-10-31 21:31:15,Alicante - Bazan,138509659,Just Eat,ACCEPTED,DELIVERY,CARD,85.39,1.99,0.0,0.0,0.0,83.4,7.57
1,2023-10-31 21:02:26,Alicante - Periodista Rodolfo Salazar,138502564,Just Eat,DELIVERED,DELIVERY,CARD,41.89,1.99,0.0,0.0,0.0,39.9,3.62
2,2023-10-29 20:20:15,Alicante - Periodista Rodolfo Salazar,138409768,Just Eat,DELIVERED,DELIVERY,CARD,31.09,1.99,0.0,0.0,0.0,29.1,2.65
3,2023-10-29 19:53:10,Alicante - Periodista Rodolfo Salazar,138402233,Just Eat,DELIVERED,DELIVERY,CARD,19.69,1.99,0.0,0.0,0.0,17.7,1.61
4,2023-10-29 19:28:17,Alicante - Periodista Rodolfo Salazar,138390234,Just Eat,DELIVERED,DELIVERY,CARD,15.99,2.99,0.0,0.0,0.0,13.0,1.19


In [23]:
deliverect.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 0 to 61
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   created_datetime   61 non-null     object 
 1   location           61 non-null     object 
 2   order_id_software  61 non-null     int64  
 3   channel            61 non-null     object 
 4   status             61 non-null     object 
 5   type               61 non-null     object 
 6   payment_method     61 non-null     object 
 7   total              61 non-null     float64
 8   delivery_cost      61 non-null     float64
 9   discount           61 non-null     float64
 10  tip                61 non-null     float64
 11  driver_tip         61 non-null     float64
 12  subtotal           61 non-null     float64
 13  vat                61 non-null     float64
dtypes: float64(7), int64(1), object(6)
memory usage: 7.1+ KB


In [24]:
deliverect = change_type(deliverect, 'order_id_software', 'int64','object')

deliverect.order_id_software.dtype

dtype('O')

In [25]:
deliverect.location.value_counts()

location
Alicante - Bazan                         32
Alicante - Periodista Rodolfo Salazar    29
Name: count, dtype: int64

In [26]:
#Asign new column to explain from which system is this order comming from

deliverect['software'] = 'deliverect'

### <a id='toc1_1_4_'></a>[DataFrames final unification](#toc0_)
The goal is to unify all columns and types and then join both of the DataFrames

In [27]:
otter.head(2)

,created_datetime,brand,location,channel,order_id_software,status,subtotal,vat,discount,software
16,2023-11-03 14:46:46,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,AULRBGNM,Entregado,26.9,0.0,0.0,otter
18,2023-11-03 14:40:15,Tepuy Burger,Tepuy Burger - Benidorm,glovo,B1YBHZJW,Entregado,51.5,0.0,0.0,otter


In [28]:
deliverect.head(2)

,created_datetime,location,order_id_software,channel,status,type,payment_method,total,delivery_cost,discount,tip,driver_tip,subtotal,vat,software
0,2023-10-31 21:31:15,Alicante - Bazan,138509659,Just Eat,ACCEPTED,DELIVERY,CARD,85.39,1.99,0.0,0.0,0.0,83.4,7.57,deliverect
1,2023-10-31 21:02:26,Alicante - Periodista Rodolfo Salazar,138502564,Just Eat,DELIVERED,DELIVERY,CARD,41.89,1.99,0.0,0.0,0.0,39.9,3.62,deliverect


#### <a id='toc1_1_4_1_'></a>[Total columns unification](#toc0_)
Column 'brand' is not in deliverect
Columns 'type','payment_method','total','delivery_cost','tip','drivertip' are not in otter. The client give the information to add

In [29]:
#New column to deliverect
deliverect['brand']= 'Tepuy Burger'
deliverect['channel']= 'just_eat'

#New columns to otter
otter['type']= 'DELIVERY'
otter['payment_method']= 'CARD'
otter['delivery_cost']= 0
otter['total']= otter.subtotal
otter['tip']= 0
otter['driver_tip']= 0

#### <a id='toc1_1_4_2_'></a>[Now we order the columns](#toc0_)

In [30]:
otter = otter[['order_id_software', 'created_datetime', 'brand', 'location', 'channel','status', 'type',  'total', 'payment_method', 'delivery_cost', 'discount', 'tip', 'driver_tip', 'subtotal', 'vat', 'software']]

In [31]:
deliverect = deliverect[['order_id_software', 'created_datetime', 'brand',  'location', 'channel', 'status', 'type', 'total', 'payment_method','delivery_cost', 'discount', 'tip', 'driver_tip', 'subtotal', 'vat', 'software']]

In [32]:
otter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 16 to 509
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id_software  60 non-null     object 
 1   created_datetime   60 non-null     object 
 2   brand              60 non-null     object 
 3   location           60 non-null     object 
 4   channel            60 non-null     object 
 5   status             60 non-null     object 
 6   type               60 non-null     object 
 7   total              60 non-null     float64
 8   payment_method     60 non-null     object 
 9   delivery_cost      60 non-null     int64  
 10  discount           60 non-null     float64
 11  tip                60 non-null     int64  
 12  driver_tip         60 non-null     int64  
 13  subtotal           60 non-null     float64
 14  vat                60 non-null     float64
 15  software           60 non-null     object 
dtypes: float64(4), int64(3), object

In [33]:
deliverect.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 0 to 61
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id_software  61 non-null     object 
 1   created_datetime   61 non-null     object 
 2   brand              61 non-null     object 
 3   location           61 non-null     object 
 4   channel            61 non-null     object 
 5   status             61 non-null     object 
 6   type               61 non-null     object 
 7   total              61 non-null     float64
 8   payment_method     61 non-null     object 
 9   delivery_cost      61 non-null     float64
 10  discount           61 non-null     float64
 11  tip                61 non-null     float64
 12  driver_tip         61 non-null     float64
 13  subtotal           61 non-null     float64
 14  vat                61 non-null     float64
 15  software           61 non-null     object 
dtypes: float64(7), object(9)
memory u

#### <a id='toc1_1_4_3_'></a>[Unification of status column](#toc0_)

In [34]:
deliverect.status.value_counts()

status
DELIVERED    57
ACCEPTED      4
Name: count, dtype: int64

In [35]:
otter.status.value_counts()

status
Entregado    54
Nuevo         5
Cancelado     1
Name: count, dtype: int64

In [36]:
otter.status = otter.status.str.upper()

In [37]:
# Change the values to english
otter.status = otter.status.apply(replace_to,  change_for='ENTREGADO', change_to='DELIVERED')
otter.status = otter.status.apply(replace_to,  change_for='NUEVO', change_to='NEW')
otter.status = otter.status.apply(replace_to,  change_for='CANCELADO', change_to='CANCELED')

#### <a id='toc1_1_4_4_'></a>[Unification of location](#toc0_)

In [38]:
otter.location.value_counts()

location
Tepuy Burger - Rodolfo Salazar    29
Tepuy Burger - Benidorm           22
Tepuy Burger - Calle Bazan         9
Name: count, dtype: int64

In [39]:
deliverect.location.value_counts()

location
Alicante - Bazan                         32
Alicante - Periodista Rodolfo Salazar    29
Name: count, dtype: int64

In [40]:
#Unify values
deliverect.location = deliverect.location.apply(replace_to,  change_for='Alicante - Bazan', change_to='Tepuy Burger - Calle Bazan')
deliverect.location = deliverect.location.apply(replace_to,  change_for='Alicante - Periodista Rodolfo Salazar', change_to='Tepuy Burger - Rodolfo Salazar')

In [41]:
all = pd.concat([otter, deliverect])

all.head()

,order_id_software,created_datetime,brand,location,channel,status,type,total,payment_method,delivery_cost,discount,tip,driver_tip,subtotal,vat,software
16,AULRBGNM,2023-11-03 14:46:46,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,DELIVERED,DELIVERY,26.9,CARD,0.0,0.0,0.0,0.0,26.9,0.0,otter
18,B1YBHZJW,2023-11-03 14:40:15,Tepuy Burger,Tepuy Burger - Benidorm,glovo,DELIVERED,DELIVERY,51.5,CARD,0.0,0.0,0.0,0.0,51.5,0.0,otter
22,ALMVLKTB,2023-11-03 14:23:06,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,DELIVERED,DELIVERY,26.9,CARD,0.0,0.0,0.0,0.0,26.9,0.0,otter
50,BXVJAQHE,2023-11-02 22:32:27,Tepuy Burger,Tepuy Burger - Benidorm,glovo,DELIVERED,DELIVERY,40.9,CARD,0.0,0.0,0.0,0.0,40.9,0.0,otter
53,ABUGGFKZ,2023-11-02 22:28:20,Tepuy Burger,Tepuy Burger - Calle Bazan,glovo,CANCELED,DELIVERY,20.4,CARD,0.0,0.0,0.0,0.0,20.4,0.0,otter


### <a id='toc1_1_5_'></a>[Export DataFrame](#toc0_)

In [42]:
all.to_csv('../../data/2-processed/data.csv', index=False)